In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/fastai-practice

# !git config --global user.email 'intaek428@gmail.com'
# !git config --global user.name 'ITHwang'

# !git add .
# !git commit -m ''
# !git push

!pip install -Uqq fastbook
from fastai.text.all import *

     |████████████████████████████████| 720 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 48 kB 5.4 MB/s 
     |████████████████████████████████| 189 kB 43.0 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 51 kB 324 kB/s 
     |████████████████████████████████| 558 kB 50.1 MB/s 
     |████████████████████████████████| 130 kB 72.1 MB/s 


ModuleNotFoundError: ignored

In [ ]:
path = untar_data(U)